# Similitud Notebook
En este notebook podremos analizar con más facilidad la similitud entre las respuestas calificadas con mayor calificación con las de menor

In [ ]:
# importo librerias
import spacy
import numpy as np
import pandas as pd
from hunspell import Hunspell

h = Hunspell('es_MX', hunspell_data_dir = 'C:/Python/Lib/site-packages/dictionaries')
spa_lex = spacy.load('es_core_news_md')

In [ ]:
# con esto evito que se impriman 'errores'
import warnings
warnings.filterwarnings("ignore")
from pprint import pprint
from IPython.display import display

### Iniciar modelo Gensim
Cristian Cardellino: Spanish Billion Words Corpus and Embeddings (March 2016), https://crscardellino.github.io/SBWCE/

In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("C:/Users/Drablaguna/Desktop/UNAM/SBW-vectors-300-min5.txt", binary=False)

### Funciones

In [ ]:
def evalChar(x):
    if x in [",",".",":",";","`","'",'"',"(",")","-","_","~","/","?","¿","=","[","]","\n","\r"]:
        x = ""
    return x
def cleanString(x):
    """eliminar caracteres de puntuacion"""
    s = map(evalChar, x)
    s = "".join(list(s))
    return s

def start_dict():
    """inicializar el diccionario de resultados, para tener resultados diferentes por prueba"""
    dic = {
    'token_base': [],      # tokens originales
    'token_toCompare': [], # token con el que se comparo
    'similarity': [],      # valor de la similitud de ambos
    'total_sim_mean': 0    # promedio similitud por frase
    }
    return dic

def checkWord(word, hun_dic, original_sentence):
    """elegir la palabra correcta entre sus sugerencias"""
    if not hun_dic.spell(word): # si la palabra NO se encuentra en el diccionario hunspell
        n = 0
        a = hun_dic.suggest(word)
        print("=> " + original_sentence)
        for x in a:
            print(str(n) + ": " + x)
            n+=1
        # se puede ingresar la respuesta correcta con el id o escribiendola manualmente
        correct_id_word = input("("+word+") ID respuesta correcta o ingresa la palabra: ")
        if correct_id_word.isdigit():
            correct = a[int(correct_id_word)]
        else:
            correct = correct_id_word
    else: # si la palabra se encuentra en el diccionario hunspell
        correct = word
    return correct.strip().lower()

def checkSentence(s_string, hun_dic):
    """checar una oracion usando la funcion de escoger la palabra correcta"""
    new_sentence = []
    for token in s_string.split(" "): # se evaluara cada token si es una palabra existente o no
        correct_word = checkWord(token, hun_dic, s_string)
        if correct_word != "": # se agregara a la oracion nueva solo si es algo
            new_sentence.append(correct_word)
    #print("\n")
    return " ".join(new_sentence)
        
# s = "porque sii mas que nada"
# s = checkSentence(s,h)
# print(s)

def print_graded_subsets(df_base, head):
    """Imprimir las dimensiones de los 4 subsets de una columna, de acuerdo a su evaluacion"""
    df_three = df_base.loc[df_base[eval_col] == 3]
    df_two   = df_base.loc[df_base[eval_col] == 2]
    df_one   = df_base.loc[df_base[eval_col] == 1]
    df_zero  = df_base.loc[df_base[eval_col] == 0]
    print(" === 3 === ")
    print(df_three.head(head))
    print(df_three.shape)
    print("\n")
    print(" === 2 === ")
    print(df_two.head(head))
    print(df_two.shape)
    print("\n")
    print(" === 1 === ")
    print(df_one.head(head))
    print(df_one.shape)
    print("\n")
    print(" === 0 === ")
    print(df_zero.head(head))
    print(df_zero.shape)

In [ ]:
def analize(base, comparer, method):
    dic = start_dict()
    
    if method == "gensim":
        base = base.split(" ")
        comparer = comparer.split(" ")
        
    for token_base in base:
        for token_toCompare in comparer:
            
            if method == "spacy":
                simil = token_base.similarity(token_toCompare)
                dic['token_base'].append(token_base.text)
                dic['token_toCompare'].append(token_toCompare.text)
                dic['similarity'].append(simil)
            
            if method == "gensim":
                try:
                    simil = model.wv.similarity(w1=token_base, w2=token_toCompare)
                except KeyError:
                    print(f"---> Alguna de las siguientes palabras: ['{token_base}', '{token_toCompare}'] , no fue encontrada.")
                    simil = 0
                dic['token_base'].append(token_base)
                dic['token_toCompare'].append(token_toCompare)
                dic['similarity'].append(simil)
                
    # genero el promedio de similitud que fue extraido de la sim de cada token con token
    dic['total_sim_mean'] = np.mean(dic['similarity'])
    return dic

def quickize(master_row, slave_row, spell_check, method):
    """analizar rapido y mostrar similitud"""
    
    """ ========== H U N S P E L L ========== """
    if spell_check == True:
        master_row = checkSentence(master_row,h)
        slave_row = checkSentence(slave_row,h)

    """ ============= S P A C Y ============= """
    if method == "spacy":
        master_row = spa_lex(master_row)
        slave_row = spa_lex(slave_row)
        dic = analize(master_row, slave_row, method)
    else:
        """ ========== G E N S I M ========== """
        dic = analize(master_row, slave_row, method)
    
    return dic

def print_zip(dic):
    """imprimir las palabras que se compararon y sus similitudes"""
    print(list(zip(dic['token_base'], dic['token_toCompare'], dic['similarity'])))

### CSV original

In [ ]:
df = pd.read_csv("C:/Users/Drablaguna/Desktop/UNAM/EvaluacionCognitiva/Bases de datos/Secundaria/SECUNDARIA_TODO.csv", encoding='latin1')

# df = pd.read_csv("C:/Users/Drablaguna/Desktop/UNAM/EvaluacionCognitiva/cuenta_de_palabras/WORDCOUNT_SECUNDARIA.csv")

df_base = df[[ans_col, eval_col]]

### Dataframes evaluados a  3, 2, 1 y 0

In [ ]:
print_graded_subsets(df_base, 10)

### Analisis

In [ ]:
def print_df_res(master_row, df_slave, ans_col):
    """evaluar 1 respuesta de 3 con master rows definidas"""
    counter = 1
    results = []
    master_row = cleanString(master_row)
    #print("============================== "+master_row.upper()+" ==============================")
    for line in df_slave[ans_col]:
        slave_row = cleanString(line)
        if slave_row != "": # para evitar evaluaciones donde la fila este vacia
            r = quickize(master_row, slave_row, False, "spacy")
            results.append(r['total_sim_mean'])
            # print(str(r['total_sim_mean'])) # SIMILITUD POR ORACION
            counter+=1
    return results

def quickPrint(master_row, df, ans_col, eval_col, n1, n2):
    df_base  = df[[ans_col, eval_col]]
    final_results = []
    
    df_three = df_base.loc[df_base[eval_col] == 3]
    df_two   = df_base.loc[df_base[eval_col] == 2]
    df_one   = df_base.loc[df_base[eval_col] == 1]
    df_zero  = df_base.loc[df_base[eval_col] == 0]

    
    # 3 con 3
    if n1 == 3 and n2 == 3:
        final_results = print_df_res(master_row, df_three, ans_col)
        
    # 3 con 2
    if n1 == 3 and n2 == 2:
        final_results = print_df_res(master_row, df_two, ans_col)
        
    # 3 con 1
    if n1 == 3 and n2 == 1:
        final_results = print_df_res(master_row, df_one, ans_col)
        
    # 3 con 0
    if n1 == 3 and n2 == 0:
        final_results = print_df_res(master_row, df_zero, ans_col)
    
    
    # 2 con 2
    if n1 == 2 and n2 == 2:
        final_results = print_df_res(master_row, df_two, ans_col)
        
    # 2 con 1
    if n1 == 2 and n2 == 1:
        final_results = print_df_res(master_row, df_one, ans_col)
        
    # 2 con 0
    if n1 == 2 and n2 == 0:
        final_results = print_df_res(master_row, df_zero, ans_col)
        
    
    # 1 con 1
    if n1 == 1 and n2 == 1:
        final_results = print_df_res(master_row, df_one, ans_col)
        
    # 1 con 0
    if n1 == 1 and n2 == 0:
        final_results = print_df_res(master_row, df_zero, ans_col)
        
    
    # 0 con 0
    if n1 == 0 and n2 == 0:
        final_results = print_df_res(master_row, df_zero, ans_col)
        
    
    # Imprimir resultados
    # pprint(list(enumerate(final_results, 1)))
    
    return final_results

### Resultados analisis

In [ ]:
""" ======================= M A S T E R  R O W S ======================= """
    
master_rows = [
    "por que hace mucho ruido",
    "porque grita mucho"
    ]
    
""" ==================================================================== """

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 3,3)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 3,2)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 3,1)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 3,0)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 2,2)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 2,1)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 2,0)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 1,1)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 1,0)

In [ ]:
quickPrint(master_row, df, ans_col, eval_col, 0,0)

### Construccion de CSV final

In [ ]:
def construct_dataframe(master_row, df, ans_col, eval_col, stopwords):
    master_name = master_row.replace(" ", "_")

    tres_tres = quickPrint(master_row, df, ans_col, eval_col, 3,3)
    tres_dos  = quickPrint(master_row, df, ans_col, eval_col, 3,2)
    tres_uno  = quickPrint(master_row, df, ans_col, eval_col, 3,1)
    tres_cero = quickPrint(master_row, df, ans_col, eval_col, 3,0)
    
    dos_dos   = quickPrint(master_row, df, ans_col, eval_col, 2,2)
    dos_uno   = quickPrint(master_row, df, ans_col, eval_col, 2,1)
    dos_cero  = quickPrint(master_row, df, ans_col, eval_col, 2,0)
    
    uno_uno   = quickPrint(master_row, df, ans_col, eval_col, 1,1)
    uno_cero  = quickPrint(master_row, df, ans_col, eval_col, 1,0)
    
    cero_cero = quickPrint(master_row, df, ans_col, eval_col, 0,0)
    
    # Longitudes de arreglos
    l3_3 = ["3_3" for x in tres_tres]
    l3_2 = ["3_2" for x in tres_dos]
    l3_1 = ["3_1" for x in tres_uno]
    l3_0 = ["3_0" for x in tres_cero]
    
    l2_2 = ["2_2" for x in dos_dos]
    l2_1 = ["2_1" for x in dos_uno]
    l2_0 = ["2_0" for x in dos_cero]
    
    l1_1 = ["1_1" for x in uno_uno]
    l1_0 = ["1_0" for x in uno_cero]
    
    l0_0 = ["0_0" for x in cero_cero]
    
    indexes_3 = l3_3 + l3_2 + l3_1 + l3_0
    indexes_2 = l2_2 + l2_1 + l2_0
    indexes_1 = l1_1 + l1_0
    
    values_3 = tres_tres + tres_dos + tres_uno + tres_cero
    values_2 = dos_dos + dos_uno + dos_cero
    values_1 = uno_uno + uno_cero

    if stopwords == True: # SI LA SIMILITUD ES CON STOPWORDS
        dic = {
            "id_3_"+master_name:    pd.Series(indexes_3),
            "eval_3_"+master_name:  pd.Series(values_3),
            "id_2_"+master_name:    pd.Series(indexes_2),
            "eval_2_"+master_name:  pd.Series(values_2),
            "id_1_"+master_name:    pd.Series(indexes_1),
            "eval_1_"+master_name:  pd.Series(values_1),
            "id_0_"+master_name:    pd.Series(l0_0),
            "eval_0_"+master_name:  pd.Series(cero_cero)
        }

        res_df = pd.DataFrame(dic, columns=[
            "id_3_"+master_name,
            "eval_3_"+master_name,
            "id_2_"+master_name,
            "eval_2_"+master_name,
            "id_1_"+master_name,
            "eval_1_"+master_name,
            "id_0_"+master_name,
            "eval_0_"+master_name
        ])
    else: # SI LA SIMILITUD ES SIN STOPWORDS
        dic = {
            "noStop_id_3_"+master_name:    pd.Series(indexes_3),
            "noStop_eval_3_"+master_name:  pd.Series(values_3),
            "noStop_id_2_"+master_name:    pd.Series(indexes_2),
            "noStop_eval_2_"+master_name:  pd.Series(values_2),
            "noStop_id_1_"+master_name:    pd.Series(indexes_1),
            "noStop_eval_1_"+master_name:  pd.Series(values_1),
            "noStop_id_0_"+master_name:    pd.Series(l0_0),
            "noStop_eval_0_"+master_name:  pd.Series(cero_cero)
        }

        res_df = pd.DataFrame(dic, columns=[
            "noStop_id_3_"+master_name,
            "noStop_eval_3_"+master_name,
            "noStop_id_2_"+master_name,
            "noStop_eval_2_"+master_name,
            "noStop_id_1_"+master_name,
            "noStop_eval_1_"+master_name,
            "noStop_id_0_"+master_name,
            "noStop_eval_0_"+master_name
        ])
    
    return res_df

Resultados de similitud todas las master rows **CON STOPWORDS**

In [ ]:
# Construccion de CSV | visualizar con -> display(stop[x])

dataframes = []

# PALABRAS CORREGIDAS SECUNDARIA ================================================================
df = pd.read_csv("C:/Users/Drablaguna/Desktop/UNAM/EvaluacionCognitiva/Bases de datos/Secundaria/palabras_corregidas_secundaria.csv", encoding='latin1')
ans_col = "clean_explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos"
eval_col = "a5"

for master in master_rows:
    dataframes.append(construct_dataframe(master, df, ans_col, eval_col, True))

Resultados de similitud todas las master rows **SIN STOPWORDS**

In [ ]:
# WORDCOUNT SECUNDARIA ===========================================================
df = pd.read_csv("C:/Users/Drablaguna/Desktop/UNAM/EvaluacionCognitiva/cuenta_de_palabras/WORDCOUNT_SECUNDARIA.csv", encoding='latin1')
ans_col = "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos_clean_ans"
eval_col = "explica_lo_que_quiere_decir_mi_hermanito_es_una_pelota_de_gritos_eval"

for master in master_rows:
    dataframes.append(construct_dataframe(master, df, ans_col, eval_col, False))

Concatenar todos los dataframes y generar el CSV final

In [ ]:
final_df = pd.concat(dataframes, axis=1)
final_df.to_csv("C:/Users/Drablaguna/Desktop/Resultados_Gensim.csv", index=False)